In [1]:
!pip install -q transformers peft bitsandbytes accelerate sentencepiece
!pip install -q llama-cpp-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 14.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

base_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base_tokenizer.pad_token = base_tokenizer.eos_token

print("Base model loaded ")

Loading base model...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Base model loaded 


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import os
import time

print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print("All libraries imported ")

CUDA Available: True
GPU: Tesla T4
All libraries imported 


In [6]:
!pip install -q --upgrade peft

In [8]:
from peft import PeftModel
import os

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

print("Merging Day 2 adapters...")
model = PeftModel.from_pretrained(model, "/content")
model = model.merge_and_unload()

print("Saving merged model...")
os.makedirs("/content/quantized", exist_ok=True)
model.save_pretrained("/content/quantized/model-fp16")
tokenizer.save_pretrained("/content/quantized/model-fp16")

print("Merged model saved ")

Loading base model...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Merging Day 2 adapters...
Saving merged model...


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Merged model saved 


In [9]:
!git clone https://github.com/ggerganov/llama.cpp
!pip install -q -r llama.cpp/requirements.txt

print("llama.cpp ready ")

Cloning into 'llama.cpp'...
remote: Enumerating objects: 81037, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 81037 (delta 1), reused 1 (delta 1), pack-reused 81035 (from 1)
Receiving objects: 100% (81037/81037), 304.87 MiB | 25.97 MiB/s, done.
Resolving deltas: 100% (58612/58612), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━

In [10]:
# Download the missing tokenizer.model file
import requests

url = "https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/tokenizer.model"
response = requests.get(url)

with open("/content/quantized/model-fp16/tokenizer.model", "wb") as f:
    f.write(response.content)

print("tokenizer.model downloaded ")

# Now convert to GGUF
!python llama.cpp/convert_hf_to_gguf.py /content/quantized/model-fp16 \
    --outfile /content/quantized/model.gguf \
    --outtype q8_0

print("GGUF model saved ")

tokenizer.model downloaded 
INFO:hf-to-gguf:Loading model: model-fp16
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:output.weight,               torch.float16 --> Q8_0, shape = {2048, 32000}
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {2048, 32000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> Q8_0, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> Q8_0, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> Q8_0, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> Q8_0, shape = {2048, 256}
INFO

In [11]:
import torch
import time

prompt = """<|system|>
If you are a doctor, please answer the medical questions based on the patient's description.</s>
<|user|>
I have been feeling very thirsty and urinating frequently. What could be wrong?</s>
<|assistant|>"""

def benchmark(name, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[1]
    torch.cuda.reset_peak_memory_stats()
    start = time.time()
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            repetition_penalty=1.3,
            pad_token_id=tokenizer.eos_token_id
        )
    end = time.time()
    latency = round(end - start, 2)
    new_tokens = output[0][input_len:]
    tokens_per_sec = round(len(new_tokens) / latency, 2)
    vram = round(torch.cuda.max_memory_allocated() / 1e9, 2)
    response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    print(f"\n{'='*50}")
    print(f"Model: {name}")
    print(f"Latency: {latency}s")
    print(f"Tokens/sec: {tokens_per_sec}")
    print(f"VRAM: {vram}GB")
    print(f"Response: {response}")
    return latency, tokens_per_sec, vram, response

# Base model
base_latency, base_tps, base_vram, base_response = benchmark("Base Model", base_model, base_tokenizer)

# Fine-tuned model
ft_latency, ft_tps, ft_vram, ft_response = benchmark("Fine-tuned Model", model, tokenizer)


Model: Base Model
Latency: 5.26s
Tokens/sec: 19.01
VRAM: 6.62GB
Response: It sounds like your body is trying to tell you that there may be an issue with hydration or bladder functioning. Here are some possible reasons for this:

1. Dehydration - If you haven't had enough water throughout the day, it can lead to dehydration which causes frequent urination due to excessive fluid loss from the kidneys. You should drink more fluids during hot weather when sweat production increases.

Model: Fine-tuned Model
Latency: 2.86s
Tokens/sec: 34.97
VRAM: 6.62GB
Response: Hi! Thanks for asking Chat Doctor . I understand your concern. Thirst is one of the most common symptoms in dehydration. It can happen due to various reasons like low blood volume or electrolyte imbalance. If it persists then there may be an underlying problem which needs further investigation by a healthcare professional. You should consult with a physician if this continues after proper hydration. In case of acute dehydration, y

In [12]:
from llama_cpp import Llama
import time

gguf = Llama(model_path="/content/quantized/model.gguf", n_ctx=512, verbose=False)

start = time.time()
gguf_output = gguf(prompt, max_tokens=100, repeat_penalty=1.3)
end = time.time()

gguf_latency = round(end - start, 2)
gguf_response = gguf_output["choices"][0]["text"].strip()
gguf_tokens = gguf_output["usage"]["completion_tokens"]
gguf_tps = round(gguf_tokens / gguf_latency, 2)
gguf_vram = round(torch.cuda.max_memory_allocated() / 1e9, 2)

print(f"Model: GGUF (Quantised)")
print(f"Latency: {gguf_latency}s")
print(f"Tokens/sec: {gguf_tps}")
print(f"VRAM: {gguf_vram}GB")
print(f"Response: {gguf_response}")

del gguf

llama_context: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized


Model: GGUF (Quantised)
Latency: 18.08s
Tokens/sec: 5.53
VRAM: 6.62GB
Response: It is possible that you have been suffering from urinary tract infection (UTI) or kidney stone. You are having frequent urination and painful urine stream, which could be due to this reason. If you are feeling weak and dizzy, it may suggest a fractured rib causing severe pain. So, make sure that your condition is normal by doing some physical examinations like blood pressure measurement, EKG (electrocardiogram) or X


In [23]:
import csv
import os

os.makedirs("/content/benchmarks", exist_ok=True)

with open("/content/benchmarks/results.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Model", "Latency(s)", "Tokens/sec", "VRAM(GB)", "Accuracy", "Response"])
    writer.writerow(["Base Model", base_latency, base_tps, base_vram, "Good", base_response])
    writer.writerow(["Fine-tuned Model", ft_latency, ft_tps, ft_vram, "Best", ft_response])
    writer.writerow(["GGUF Model", gguf_latency, gguf_tps, "0.0 (CPU)", "Good", gguf_response])

print("results.csv fixed ")

results.csv fixed 


In [14]:
from transformers import TextStreamer

# Base Model
print(" Base Model Streaming:")
print("="*40)
inputs = base_tokenizer(prompt, return_tensors="pt").to(base_model.device)
streamer = TextStreamer(base_tokenizer, skip_prompt=True, skip_special_tokens=True)
with torch.no_grad():
    base_model.generate(**inputs, max_new_tokens=100, repetition_penalty=1.3, pad_token_id=base_tokenizer.eos_token_id, streamer=streamer)

# Fine-tuned Model
print("\n Fine-tuned Model Streaming:")
print("="*40)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
with torch.no_grad():
    model.generate(**inputs, max_new_tokens=100, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id, streamer=streamer)

# GGUF Model
print("\n GGUF Model Streaming:")
print("="*40)
gguf = Llama(model_path="/content/quantized/model.gguf", n_ctx=512, verbose=False)
for chunk in gguf(prompt, max_tokens=100, repeat_penalty=1.3, stream=True):
    print(chunk["choices"][0]["text"], end="", flush=True)
del gguf

print("\n\nStreaming done ")

 Base Model Streaming:

It sounds like your body is trying to tell you that there may be an issue with hydration or bladder functioning. Here are some possible reasons for this:

1. Dehydration - If you haven't had enough water throughout the day, it can lead to dehydration which causes frequent urination due to excessive fluid loss from the kidneys. You should drink more fluids during hot weather when sweat production increases.


 Fine-tuned Model Streaming:

Hi! Thanks for asking Chat Doctor . I understand your concern. Thirst is one of the most common symptoms in dehydration. It can happen due to various reasons like low blood volume or electrolyte imbalance. If it persists then there may be an underlying problem which needs further investigation by a healthcare professional. You should consult with a physician if this continues after proper hydration. In case of acute dehydration, you

 GGUF Model Streaming:


llama_context: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized



It is possible that you have been suffering from urinary tract infection (UTI) or kidney stone. You are having frequent urination and painful urine stream, which could be due to this reason. If you are feeling weak and dizzy, it may suggest a fractured rib causing severe pain. So, make sure that your condition is normal by doing some physical examinations like blood pressure measurement, EKG (electrocardiogram) or X

Streaming done 


In [15]:
prompts = [
    "What is diabetes?",
    "What are the symptoms of high blood pressure?",
    "How to treat a fever?"
]

def batch_inference(name, model, tokenizer):
    print(f"\n Batch Inference — {name}:")
    print("="*40)
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)
    start = time.time()
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id)
    batch_time = round(time.time() - start, 2)
    for i, output in enumerate(outputs):
        response = tokenizer.decode(output, skip_special_tokens=True)
        print(f"\nPrompt {i+1}: {prompts[i]}")
        print(f"Response: {response}")
    print(f"\nTotal batch time: {batch_time}s ")

# Base Model
batch_inference("Base Model", base_model, base_tokenizer)

# Fine-tuned Model
batch_inference("Fine-tuned Model", model, tokenizer)

print(" Batch Inference — GGUF Model (Fixed):")
print("="*40)
gguf = Llama(model_path="/content/quantized/model.gguf", n_ctx=512, verbose=False)
start = time.time()
for i, p in enumerate(prompts):
    full_prompt = f"Question: {p}\nAnswer:"
    response = gguf(full_prompt, max_tokens=80, repeat_penalty=1.3)["choices"][0]["text"].strip()
    print(f"\nPrompt {i+1}: {p}")
    print(f"Response: {response}")
print(f"\nTotal batch time: {round(time.time() - start, 2)}s ")
del gguf

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



 Batch Inference — Base Model:


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Prompt 1: What is diabetes?
Response: What is diabetes? 
- Answer: Diabetes is a chronic disease that occurs when the body does not produce or use insulin properly. Insulin helps cells in the body absorb glucose (sugar) from food, and

Prompt 2: What are the symptoms of high blood pressure?
Response: What are the symptoms of high blood pressure?

Prompt 3: How to treat a fever?
Response: How to treat a fever? 
- What are the symptoms of influenza and how can they be treated? 
- When should I seek medical attention for an ear infection, and what treatment options do I have available at home? 
- Which vacc

Total batch time: 2.25s 

 Batch Inference — Fine-tuned Model:


llama_context: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized



Prompt 1: What is diabetes?
Response: What is diabetes? 
If you have been diagnosed with type 2 diabetes, it means that your body has developed a resistance to insulin. Insulin helps the cells in our bodies absorb glucose (sugar) from

Prompt 2: What are the symptoms of high blood pressure?
Response: What are the symptoms of high blood pressure?
What is a normal BP reading for an adult male, and what should I do if it's above 140/90 mm Hg or below 85/60 mm Hg?

Prompt 3: How to treat a fever?
Response: How to treat a fever? 
If you are feeling unwell, take care of yourself by drinking plenty of fluids and eating nutritious foods. If your temperature is above or below normal range (over/under), seek medical attention immediately. Avoid

Total batch time: 1.39s 
 Batch Inference — GGUF Model (Fixed):

Prompt 1: What is diabetes?
Response: Diabetes is a disease in which the body cannot produce or use insulin effectively. Insulin helps glucose (sugar) to enter cells for energy and stores 

In [16]:
multi_prompts = [
    "What is insulin?",
    "What causes high cholesterol?",
    "How to control blood sugar?",
    "What are symptoms of heart attack?",
    "How to reduce stress?"
]

def multi_prompt_test(name, model, tokenizer):
    print(f"\n Multi-prompt Test — {name}:")
    print("="*40)
    for i, p in enumerate(multi_prompts):
        inputs = tokenizer(p, return_tensors="pt").to(model.device)
        input_len = inputs["input_ids"].shape[1]
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=50, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(output[0][input_len:], skip_special_tokens=True).strip()
        print(f"\nPrompt {i+1}: {p}")
        print(f"Response: {response}")

# Base Model
multi_prompt_test("Base Model", base_model, base_tokenizer)

# Fine-tuned Model
multi_prompt_test("Fine-tuned Model", model, tokenizer)

# GGUF Model
print("\n Multi-prompt Test — GGUF Model:")
print("="*40)
gguf = Llama(model_path="/content/quantized/model.gguf", n_ctx=512, verbose=False)
for i, p in enumerate(multi_prompts):
    response = gguf(f"Question: {p}\nAnswer:", max_tokens=50, repeat_penalty=1.3)["choices"][0]["text"].strip()
    print(f"\nPrompt {i+1}: {p}")
    print(f"Response: {response}")
del gguf

print("\nMulti-prompt test done ")


 Multi-prompt Test — Base Model:

Prompt 1: What is insulin?
Response: How does it work in the body and what are its functions?

Prompt 2: What causes high cholesterol?
Response: 

Prompt 3: How to control blood sugar?
Response: How does the body regulate glucose levels in response to insulin action and what are some potential consequences of abnormal glycemic control?

Prompt 4: What are symptoms of heart attack?
Response: 

Prompt 5: How to reduce stress?
Response: 

 Multi-prompt Test — Fine-tuned Model:

Prompt 1: What is insulin?
Response: What are the different types of diabetes and how do they affect a person's body?

Prompt 2: What causes high cholesterol?
Response: What are the symptoms of high blood pressure in women and how can it be treated?

Prompt 3: How to control blood sugar?
Response: I have been diagnosed with type 2 diabetes. I am on medication and exercise regularly, but my levels are still high (140/85). What can I do differently to lower them further?


llama_context: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized



Prompt 4: What are symptoms of heart attack?
Response: What is the difference between a coronary artery disease and an angina pectoris?

Prompt 5: How to reduce stress?
Response: 

 Multi-prompt Test — GGUF Model:

Prompt 1: What is insulin?
Response: Insulin is a hormone produced by the pancreas and released into the bloodstream to control glucose levels in our body.

Prompt 2: What causes high cholesterol?
Response: Cholesterol is a fatty substance that is necessary for normal growth and development in the body. It also helps to build tissues, proteins (such as collagen), and carries nutrients throughout your bloodstream. However

Prompt 3: How to control blood sugar?
Response: Hi, I'm using glucose meter for my blood sugar control and it is giving me reading as 140. But when I take insulin in morning with breakfast (daily) then the number goes down to 6

Prompt 4: What are symptoms of heart attack?
Response: Heart attack is a medical emergency. Here are the common symptoms:
1) Ches

In [17]:
import os
os.makedirs("/content/inference", exist_ok=True)

code = '''
import torch, time, os
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from llama_cpp import Llama

# Config
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
FINETUNED_MODEL = "/content/quantized/model-fp16" if os.path.exists("/content") else "./quantized/model-fp16"
GGUF_MODEL = "/content/quantized/model.gguf" if os.path.exists("/content") else "./quantized/model.gguf"

PROMPT = """<|system|>
If you are a doctor, please answer the medical questions based on the patient description.</s>
<|user|>
I have been feeling very thirsty and urinating frequently. What could be wrong?</s>
<|assistant|>"""

BATCH_PROMPTS = ["What is diabetes?", "What are symptoms of high blood pressure?", "How to treat fever?"]
MULTI_PROMPTS = ["What is insulin?", "What causes high cholesterol?", "How to control blood sugar?", "What are symptoms of heart attack?", "How to reduce stress?"]

# Load HuggingFace Model
def load_model(path):
    model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.float16, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(path)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Generate response
def generate(model, tokenizer, prompt, max_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[1]
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=max_tokens, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0][input_len:], skip_special_tokens=True).strip(), len(output[0][input_len:])

# Benchmark
def benchmark(name, model, tokenizer):
    print(f"\\n=== Benchmark: {name} ===")
    torch.cuda.reset_peak_memory_stats()
    start = time.time()
    response, num_tokens = generate(model, tokenizer, PROMPT)
    latency = round(time.time() - start, 2)
    print(f"Latency: {latency}s | Tokens/sec: {round(num_tokens/latency, 2)} | VRAM: {round(torch.cuda.max_memory_allocated()/1e9, 2)}GB")
    print(f"Response: {response}")

def benchmark_gguf(gguf):
    print("\\n=== Benchmark: GGUF ===")
    start = time.time()
    out = gguf(PROMPT, max_tokens=100, repeat_penalty=1.3)
    latency = round(time.time()-start, 2)
    print(f"Latency: {latency}s | Tokens/sec: {round(out['usage']['completion_tokens']/latency, 2)} | VRAM: 0.0GB (CPU based)")
    print(f"Response: {out['choices'][0]['text'].strip()}")

# Streaming
def streaming(name, model, tokenizer):
    print(f"\\n=== Streaming: {name} ===")
    inputs = tokenizer(PROMPT, return_tensors="pt").to(model.device)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    with torch.no_grad():
        model.generate(**inputs, max_new_tokens=100, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id, streamer=streamer)

def streaming_gguf(gguf):
    print("\\n=== Streaming: GGUF ===")
    for chunk in gguf(PROMPT, max_tokens=100, repeat_penalty=1.3, stream=True):
        print(chunk["choices"][0]["text"], end="", flush=True)
    print()

# Batch Inference
def batch_inference(name, model, tokenizer):
    print(f"\\n=== Batch Inference: {name} ===")
    inputs = tokenizer(BATCH_PROMPTS, return_tensors="pt", padding=True, truncation=True).to(model.device)
    start = time.time()
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id)
    print(f"Batch time: {round(time.time()-start, 2)}s")
    for i, out in enumerate(outputs):
        print(f"Q{i+1}: {BATCH_PROMPTS[i]} -> {tokenizer.decode(out, skip_special_tokens=True)}")

def batch_inference_gguf(gguf):
    print("\\n=== Batch Inference: GGUF ===")
    start = time.time()
    for i, p in enumerate(BATCH_PROMPTS):
        print(f"Q{i+1}: {p} -> {gguf(f'Question: {p} Answer:', max_tokens=50, repeat_penalty=1.3)['choices'][0]['text'].strip()}")
    print(f"Batch time: {round(time.time()-start, 2)}s")

# Multi-prompt Test
def multi_prompt_test(name, model, tokenizer):
    print(f"\\n=== Multi-prompt: {name} ===")
    for i, p in enumerate(MULTI_PROMPTS):
        response, _ = generate(model, tokenizer, p, max_tokens=50)
        print(f"Q{i+1}: {p} -> {response}")

def multi_prompt_test_gguf(gguf):
    print("\\n=== Multi-prompt: GGUF ===")
    for i, p in enumerate(MULTI_PROMPTS):
        print(f"Q{i+1}: {p} -> {gguf(f'Question: {p} Answer:', max_tokens=50, repeat_penalty=1.3)['choices'][0]['text'].strip()}")

# Main
if __name__ == "__main__":
    # Base Model
    base_model, base_tokenizer = load_model(BASE_MODEL)
    benchmark("Base Model", base_model, base_tokenizer)
    streaming("Base Model", base_model, base_tokenizer)
    batch_inference("Base Model", base_model, base_tokenizer)
    multi_prompt_test("Base Model", base_model, base_tokenizer)

    # Fine-tuned Model
    ft_model, ft_tokenizer = load_model(FINETUNED_MODEL)
    benchmark("Fine-tuned Model", ft_model, ft_tokenizer)
    streaming("Fine-tuned Model", ft_model, ft_tokenizer)
    batch_inference("Fine-tuned Model", ft_model, ft_tokenizer)
    multi_prompt_test("Fine-tuned Model", ft_model, ft_tokenizer)

    # GGUF Model
    gguf = Llama(model_path=GGUF_MODEL, n_ctx=512, verbose=False)
    benchmark_gguf(gguf)
    streaming_gguf(gguf)
    batch_inference_gguf(gguf)
    multi_prompt_test_gguf(gguf)
    del gguf

    print("\\nAll done ")
'''

with open("/content/inference/test_inference.py", "w") as f:
    f.write(code)

print("test_inference.py created ")

test_inference.py created 


In [ ]:
# Fix the torch conflict first
!pip install -q torchvision --upgrade
!pip install -q transformers --upgrade

# Now run the script
!python /content/inference/test_inference.py

`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100% 201/201 [00:10<00:00, 20.06it/s, Materializing param=model.norm.weight]

=== Benchmark: Base Model ===
Latency: 4.23s | Tokens/sec: 23.64 | VRAM: 2.22GB
Response: It sounds like you may need to consult with your healthcare provider for further evaluation or treatment options. Here's what I can tell you:

1. Thirstiness - This is often caused by dehydration due to excessive sweat production during hot weather. You should drink plenty of fluids (water, soda, tea) throughout the day to help replenish lost water content in your body. If this doesn't alleviate symptoms

=== Streaming: Base Model ===

It sounds like you may need to consult with your healthcare provider for further evaluation or treatment options. Here's what I can tell you:

1. Thirstiness - This is often caused by dehydration due to excessive sweat production during hot weather. You should drink plenty of fluids (water, soda, tea) throughout the day to 